## ReMKiT1D input generator - CVODE and DerivationTerms

This notebook generates an input file for a ReMKiT1D run solving the predator-prey system and showcasing new v1.2.0 features such as the CVODE integrator support and DerivationTerms.

The predator-prey system can be written as follows:

$$ \frac{dx}{dt} = (\alpha - \beta y)x $$
$$ \frac{dy}{dt} = (\delta x - \gamma)y $$

where $\alpha$ and $\beta$ are the prey growth and death factors, and $\delta$ and $\gamma$ are the predator growth and death factors. $x$ and $y$ are prey and predator numbers in arbitrary units.

In [ ]:
from RMK_support import RKWrapper ,Grid, Node, treeDerivation
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io
import RMK_support.dashboard_support as ds
import RMK_support.common_models as cm

import numpy as np
import holoviews as hv 
import panel as pn
import matplotlib.pyplot as plt

Wrapper and grid setup. The grid is 0D in this simple problem.

In [ ]:
rk = RKWrapper(addTimeVar=True) # The new default, adding the time variable the first time any variable is added
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/RMK_pred_prey/"
rk.setHDF5Path(hdf5Filepath) 
rk.grid = Grid(np.zeros(1))

Setting initial x and y values, and setting two pairs of predatorr-prey variables, one to be evolved with RK2 and the other with CVODE.

In [ ]:
x0 = 10*np.ones(1)
y0 = 2 *np.ones(1)

rk.addVar("xVar1",x0)
rk.addVar("yVar1",y0)
rk.addVar("xVar2",x0)
rk.addVar("yVar2",y0)

alpha = 1.1
beta = 0.4
delta = 0.1
gamma = 0.4

In order to use DerivationTerms, we register two pairs of derivations using the calculation tree approach.

In [ ]:
rk.addCustomDerivation("xDeriv1",derivOptions=treeDerivation(alpha*Node("xVar1")-beta*Node("yVar1")*Node("xVar1")))
rk.addCustomDerivation("yDeriv1",derivOptions=treeDerivation(delta*Node("xVar1")*Node("yVar1")-gamma*Node("yVar1")))

rk.addCustomDerivation("xDeriv2",derivOptions=treeDerivation(alpha*Node("xVar2")-beta*Node("yVar2")*Node("xVar2")))
rk.addCustomDerivation("yDeriv2",derivOptions=treeDerivation(delta*Node("xVar2")*Node("yVar2")-gamma*Node("yVar2")))

We now add 4 models, one for each evolved variable, and each with a single derivation term.

In [ ]:
xModel = sc.CustomModel("xEvo1")

xModel.addTerm("_term",sc.DerivationTerm("xVar1",sc.derivationRule("xDeriv1",["xVar1","yVar1"])))

rk.addModel(xModel)

yModel = sc.CustomModel("yEvo1")

yModel.addTerm("_term",sc.DerivationTerm("yVar1",sc.derivationRule("yDeriv1",["xVar1","yVar1"]),generalGroups=[2])) # We set the group to 2 here just to demonstrate that term groups are now automatically handled when setting integrator global options below

rk.addModel(yModel)

In [ ]:
xModel = sc.CustomModel("xEvo2")

xModel.addTerm("_term",sc.DerivationTerm("xVar2",sc.derivationRule("xDeriv2",["xVar2","yVar2"])))

rk.addModel(xModel)

yModel = sc.CustomModel("yEvo2")

yModel.addTerm("_term",sc.DerivationTerm("yVar2",sc.derivationRule("yDeriv2",["xVar2","yVar2"]),generalGroups=[2]))

rk.addModel(yModel)

### Integrator setup 

In [ ]:
rk.setIntegratorGlobalData(initialTimestep=0.01) # Note that we did not have to set group numbers
rk.addIntegrator("CVODE",sc.CVODEIntegrator(relTol=1e-5,absTol=1e-12)) # Try out different CVODE options (for example relTol = 1e-5 will significantly reduce accuracy)

integrationStep = sc.IntegrationStep("CVODE")

# We add the first two models to the CVODE integration
for tag in rk.modelTags()[:2]:
    # Here we can automatically detect which groups are active for the given term (this handles both implicit and general groups)
    integrationStep.addModel(tag,updateGroups=rk.activeGroups(tag),evaluateGroups=rk.activeGroups(tag))

rk.addIntegrationStep("step1",integrationStep.dict())

rk.addIntegrator("RK2",sc.rkIntegrator(2)) # Try out different orders here, too (first order clearly cannot handle the problem)

integrationStep = sc.IntegrationStep("RK2")

#The other two models go in the RK2 integrator step
for tag in rk.modelTags()[2:]:
    integrationStep.addModel(tag,updateGroups=rk.activeGroups(tag),evaluateGroups=rk.activeGroups(tag))

rk.addIntegrationStep("step2",integrationStep.dict())

In [ ]:
rk.setTimeTargetTimestepping(50.0) # Run until 50 time units have elapsed
rk.setMinimumIntervalOutput(0.1) # Output every 0.1 time unit (might not have exactly 500 files because of rounding)

### Output configuration file

In [ ]:
rk.writeConfigFile()

## Data analysis

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=100,dpi=150)

numFiles=471
loadpath = rk.hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont, filepaths=loadFilenames)
loadedData

### Comparing the two solvers

In [ ]:
pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)

# Time trace comparison
dashboard.fluidMultiComparison(["xVar1","yVar1","xVar2","yVar2"],fixedPosition=True)

Orbit comparison

In [ ]:
fig,ax = plt.subplots(1,2,sharey="row")

ax[0].plot(loadedData["xVar2"],loadedData["yVar2"])

ax[0].title.set_text("RK2")

ax[0].set_xlabel("x")
ax[0].set_ylabel("y")

ax[1].plot(loadedData["xVar1"],loadedData["yVar1"])

ax[1].title.set_text("CVODE")

ax[1].set_xlabel("x")